In [1]:
import pandas as pd
df = pd.read_csv("inference_data_indices.csv")
df['eventDate'] = pd.to_datetime(df['eventDate'], format='%Y-%m-%d', errors='coerce')

In [2]:
winter_wheat = pd.read_csv("winter wheat.csv", delimiter='\t')
winter_wheat = winter_wheat[winter_wheat["occurrenceStatus"] == "PRESENT"]
winter_wheat = winter_wheat[winter_wheat['year'] >= 2022]
winter_wheat['eventDate'] = pd.to_datetime(winter_wheat['eventDate'], format='%Y-%m-%d', errors='coerce')
winter_wheat = winter_wheat[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
winter_wheat = winter_wheat.dropna().reset_index(drop=True)


/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_79347/2925844888.py:1: DtypeWarning: Columns (39,41,46) have mixed types. Specify dtype option on import or set low_memory=False.
  winter_wheat = pd.read_csv("winter wheat.csv", delimiter='\t')


In [4]:
import numpy as np
from sklearn.neighbors import BallTree
from tqdm import tqdm
# Convert lat/lon to radians for BallTree
brown_hare_coords = np.deg2rad(df[['decimalLatitude', 'decimalLongitude']].values)
winter_wheat_coords = np.deg2rad(winter_wheat[['decimalLatitude', 'decimalLongitude']].values)

# Build BallTree using haversine metric
tree = BallTree(winter_wheat_coords, metric='haversine')

# Define search radius: 1 km in radians
earth_radius_km = 6371.0
radius = 1 / earth_radius_km  # 1 km in radians

# Initialize presence columns
df['winter_wheat_presence'] = 0

# Iterate through each brown hare point with tqdm for progress tracking
for i in tqdm(range(len(df))):
    point = brown_hare_coords[i].reshape(1, -1)
    event_month = df.loc[i, 'eventDate'].month
    event_year = df.loc[i, 'eventDate'].year

    # WINTER WHEAT
    idxs = tree.query_radius(point, r=radius)[0]
    for j in idxs:
        winter_wheat_date = winter_wheat.loc[j, 'eventDate']
        if winter_wheat_date.month == event_month and winter_wheat_date.year == event_year:
            df.at[i, 'winter_wheat_presence'] = 1
            break  # Found at least one match, no need to check further

100%|██████████| 244359/244359 [00:08<00:00, 29175.87it/s]


In [ ]:
from pyproj import Transformer
from rasterio.windows import Window
import rasterio
land_cover_map = {
    1: "Deciduous woodland",
    2: "Coniferous woodland",
    3: "Arable",
    4: "Improved grassland",
    5: "Neutral grassland",
    6: "Calcareous grassland",
    7: "Acid grassland",
    8: "Fen",
    9: "Heather",
    10: "Heather grassland",
    11: "Bog",
    12: "Inland rock",
    13: "Saltwater",
    14: "Freshwater",
    15: "Supralittoral rock",
    16: "Supralittoral sediment",
    17: "Littoral rock",
    18: "Littoral sediment",
    19: "Saltmarsh",
    20: "Urban",
    21: "Suburban"
}

# Batch coordinate transformation
transformer_ni = Transformer.from_crs("EPSG:4326", "EPSG:29903", always_xy=True)
transformer_gb = Transformer.from_crs("EPSG:4326", "EPSG:27700", always_xy=True)

coords = list(zip(df['decimalLongitude'], df['decimalLatitude']))
df['easting_ni'], df['northing_ni'] = zip(*transformer_ni.itransform(coords))
df['easting_gb'], df['northing_gb'] = zip(*transformer_gb.itransform(coords))

# Raster processing optimization
gb_raster = 'gblcm2023_10m.tif'
n_ireland_raster = 'nilcm2023_10m.tif'


def get_land_cover_class(row):
    try:
        # Try GB raster first
        with rasterio.open(gb_raster) as src:
            row_idx, col_idx = src.index(row['easting_gb'], row['northing_gb'])
            # Read a small window around the point for better performance
            window = Window(col_idx, row_idx, 1, 1)
            land_cover_class = src.read(1, window=window)[0, 0]

            if land_cover_class == 0:  # Check NI raster if GB is 0
                with rasterio.open(n_ireland_raster) as src_ni:
                    row_idx, col_idx = src_ni.index(row['easting_ni'], row['northing_ni'])
                    window = Window(col_idx, row_idx, 1, 1)
                    land_cover_class = src_ni.read(1, window=window)[0, 0]

        return land_cover_map.get(land_cover_class, "Unknown")
    except Exception as e:
        print(f"Error processing row: {e}")
        return "Unknown"


tqdm.pandas()
df['Land_cover'] = df.progress_apply(get_land_cover_class, axis=1)


  0%|          | 1119/244359 [00:02<08:15, 491.33it/s]

Error processing row: index 0 is out of bounds for axis 0 with size 0


 19%|█▉        | 46182/244359 [01:42<07:02, 469.55it/s]

Error processing row: index 0 is out of bounds for axis 0 with size 0


 32%|███▏      | 79287/244359 [02:59<06:17, 437.00it/s]

Error processing row: index 0 is out of bounds for axis 0 with size 0


 61%|██████    | 148877/244359 [06:22<04:28, 355.26it/s]

Error processing row: index 0 is out of bounds for axis 1 with size 0


 69%|██████▉   | 168885/244359 [07:50<06:19, 198.73it/s]

In [ ]:
df = df.dropna()
df=df.drop(['easting_ni', 'northing_ni', 'easting_gb', 'northing_gb'],axis=1)
# Remove where Land_cover is Unknown
df = df[df['Land_cover'] != "Unknown"].reset_index(drop=True)
df.to_csv("brown_hare_inference_data_preprocessed.csv")
